In [1]:
import numpy as np
import pandas as pd

from parse_input_file import parse_file
from create_feature_dfs import compose_signals_df
from LeastSquaresMethod import LeastSquaresMethod
from RecursiveLeastSquaresMethod import RecursiveLeastSquaresMethod
from metrics import count_model_coef_rmse, rsse, determination_coef

In [2]:
parsing_result = parse_file('Test+')
features_dfs = compose_signals_df(parsing_result)

In [3]:
features_dfs.head()

,y(k),y(k-1),y(k-2),y(k-3),v(k),v(k-1),v(k-2),v(k-3)
0,-0.129424,0.012543,-0.330653,-0.263688,0.789293,0.294494,-1.258052,-0.178062
1,-0.097545,-0.129424,0.012543,-0.330653,0.267935,0.789293,0.294494,-1.258052
2,-0.027886,-0.097545,-0.129424,0.012543,0.463723,0.267935,0.789293,0.294494
3,-0.046153,-0.027886,-0.097545,-0.129424,0.984378,0.463723,0.267935,0.789293
4,-0.026714,-0.046153,-0.027886,-0.097545,-0.551910,0.984378,0.463723,0.267935


# Least Square Method

In [4]:
ls_method = LeastSquaresMethod(biased=True)

In [5]:
ls_method.fit(X=np.array(features_dfs.drop(columns='''y(k)''')), 
              y=np.array(features_dfs['''y(k)''']))

In [6]:
count_model_coef_rmse(ls_method, parsing_result)

1.0106898154882096

In [7]:
ls_pred = ls_method.predict(np.array(features_dfs.drop(columns='''y(k)''')))

In [9]:
print({
    'rsse': rsse(np.array(features_dfs['''y(k)''']), ls_pred),
    'determination_coef': determination_coef(np.array(features_dfs['''y(k)''']), ls_pred)
})

{'rsse': 11.306348368472845, 'determination_coef': 0.8691696792706486}


# Recursive Least Square Method

In [10]:
num_features = max(max(parsing_result['out_weights'].keys()), max(parsing_result['manage_weights'].keys()))*2 + 1

In [11]:
rls_method = RecursiveLeastSquaresMethod(coef_matrix_shape=num_features, biased=True)

In [13]:
rls_pred = rls_method.predict_all(X=np.array(features_dfs.drop(columns='''y(k)''')), 
                                  y=np.array(features_dfs['''y(k)''']))

In [14]:
count_model_coef_rmse(rls_method, parsing_result)

1.0046684387758302

In [15]:
print({
    'rsse': rsse(np.array(features_dfs['''y(k)''']), rls_pred),
    'determination_coef': determination_coef(np.array(features_dfs['''y(k)''']), rls_pred)
})

{'rsse': 0.012219110091415759, 'determination_coef': 0.7071789581664429}
